In [1]:
import torch
from sde_gp_em import jacobian_of_drift

In [2]:
from sdes import SDE,ConstantDiffusion,TwoDimensionalSynDrift,DoubleWellDrift,MaxDiffusion

In [25]:
dt = 0.002
number_of_steps = 10000
num_paths = 3
sigma = 1.

x_0 = torch.rand((1,)).unsqueeze(-1)
real_drift = DoubleWellDrift(a=4.,b=4.)
diffusion = MaxDiffusion()
#diffusion = ConstantDiffusion(sigma=sigma)

#x_0 = torch.rand((2,)).unsqueeze(0)
#real_drift = TwoDimensionalSynDrift()
#diffusion = ConstantDiffusion(sigma=sigma)

sde = SDE(real_drift,diffusion,dt=dt)
dense_path_realization = sde.simulate(x_0, number_of_steps, num_paths=num_paths)
dense_path_realization = dense_path_realization[0,:,:]

In [26]:
#=============================================
#DEFINE SPARSE OBSERVATIONS
#=============================================
num_dense_steps_in_bridge = 2
number_of_steps = dense_path_realization.size(0)
dense_time = torch.arange(0,number_of_steps)*dt
max_time = number_of_steps*dt
observation_index = range(0,number_of_steps,num_dense_steps_in_bridge)
sparse_observation_time = dense_time[observation_index]
sparse_observations = dense_path_realization[observation_index]

In [27]:
n_points = sparse_observations.size(0)
dimensions = sparse_observations.size(1)
observations = sparse_observations

In [36]:
diffusion_diagonal = diffusion(observations)
drift_at_points = real_drift(observations)
Gamma = jacobian_of_drift(real_drift,observations)

D = torch.zeros((n_points,dimensions,dimensions))
E = torch.zeros((n_points,2*dimensions,2*dimensions))
OI = torch.zeros((2*dimensions,dimensions))
OI[dimensions:,:] = torch.eye(dimensions)

D[:,range(dimensions),range(dimensions)] = diffusion_diagonal
E[:,:dimensions,:dimensions] = Gamma 
E[:,:dimensions:,dimensions:] = D
E[:,dimensions:,dimensions:] = Gamma.transpose(2,1)

In [37]:
Gamma.shape

torch.Size([5001, 1, 1])

In [30]:
drift_at_points.shape

torch.Size([5001, 1])

In [38]:
import torch

def jacobian_fd_batch_no_loop(f, x, epsilon=1e-6):
    """
    Compute the Jacobian of the function f: R^3 -> R^3 using finite differences for a batch of points,
    without using explicit loops.
    
    Args:
    - f (callable): the function mapping R^3 to R^3
    - x (torch.Tensor): input tensor of shape (number_of_points, 3)
    - epsilon (float): small perturbation value for finite differences

    Returns:
    - J (torch.Tensor): Jacobian matrix for each point, shape (number_of_points, 3, 3)
    """
    num_points = x.shape[0]
    dimensions = x.shape[1]
    
    # Compute the function value for all points
    fx = f(x)  # Shape: (number_of_points, dimensions)
    
    # Prepare perturbations for all dimensions (broadcasting approach)
    perturbations = torch.eye(dimensions, device=x.device).unsqueeze(0) * epsilon  # Shape: (1, dimensions, dimensions)
    x_perturbed = x.unsqueeze(1) + perturbations  # Shape: (number_of_points, dimensions, dimensions)
    
    # Reshape x_perturbed to apply f
    x_perturbed_flat = x_perturbed.view(-1, dimensions)  # Shape: (number_of_points * dimensions, dimensions)
    
    # Compute f(x + epsilon * e_i) for all perturbed points
    fx_perturbed = f(x_perturbed_flat).view(num_points, dimensions, dimensions)  # Shape: (number_of_points, dimensions, dimensions)
    
    # Compute the Jacobian using finite differences
    J = (fx_perturbed - fx.unsqueeze(2)) / epsilon  # Shape: (number_of_points, dimensions, dimensions)
    
    return J

# Example usage
# Define a function f: R^3 -> R^3
def f(x):
    return torch.stack([x[:, 0]**2, x[:, 1]**3, torch.sin(x[:, 2])], dim=1)

# Define a batch of input points (2 points in this example)
x = torch.tensor([[1.0],
                  [0.5]], requires_grad=True)

In [39]:
# Compute the Jacobian for each point
J_batch = jacobian_fd_batch_no_loop(real_drift, x,epsilon=1e-5)
print(J_batch)

tensor([[[-8.0109]],

        [[ 1.0014]]], grad_fn=<DivBackward0>)


In [42]:
Gamma = jacobian_of_drift(real_drift,x)
Gamma.shape

torch.Size([2, 1, 1])

In [41]:
J_batch.shape

torch.Size([2, 1, 1])